# Exportar registros y enviar correos de forma local

Este cuaderno permite ejecutar paso a paso la exportación de los participantes y respuestas de formularios de ODK Central. También se puede filtrar a los participantes y enviarles un correo utilizando el mismo sistema que `export_local.py`.

In [ ]:
import os
import asyncio
import requests
import xmltodict
from typing import Dict, Any, List, Callable

from search_by_odk_api import get_odk_token
from mailer import send_email

## Funciones de apoyo

In [ ]:
def _export_participants(base_url: str, token: str, project_id: int) -> List[Dict[str, Any]]:
    """Return all entities from the `participantes` dataset."""
    headers = {"Authorization": f'Bearer {token}'}
    url = f'{base_url}/v1/projects/{project_id}/datasets/participantes/entities'
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

def _export_form_submissions(base_url: str, token: str, project_id: int, form_id: str) -> List[Dict[str, Any]]:
    """Return all submissions for a given form as parsed dicts."""
    headers = {"Authorization": f'Bearer {token}'}
    list_url = f'{base_url}/v1/projects/{project_id}/forms/{form_id}/submissions'
    resp = requests.get(list_url, headers=headers)
    resp.raise_for_status()
    submissions = []
    for sub in resp.json():
        xml_url = f'{list_url}/{sub["instanceId"]}.xml'
        xml_resp = requests.get(xml_url, headers=headers)
        if xml_resp.status_code != 200:
            continue
        parsed = xmltodict.parse(xml_resp.text)
        submissions.append(parsed.get('data', {}))
    return submissions

## 1. Obtener token y exportar datos

In [ ]:
base_url = os.getenv('ODK_BASE_URL', '').rstrip('/')
project_id = int(os.getenv('ODK_PROJECT_ID', '2'))

token = get_odk_token()
if not token:
    raise RuntimeError('No se pudo obtener token')

participants = _export_participants(base_url, token, project_id)

consent_form_id = os.getenv('ODK_FORM_CONSENT', 'Laura2-piloto-encuesta-ic')
form_ids = [
    os.getenv('ODK_FORM_PREREGISTRO'),
    os.getenv('ODK_FORM_EN_P1'),
    os.getenv('ODK_FORM_EN_P2'),
    os.getenv('ODK_FORM_EN_P3'),
    consent_form_id,
]

forms_data = {}
for fid in form_ids:
    if fid:
        forms_data[fid] = _export_form_submissions(base_url, token, project_id, fid)

print(f'✔️ Participantes: {len(participants)} registros')
for fid in forms_data:
    print(f'✔️ Form {fid}: {len(forms_data[fid])} submissions')

## 2. Definir función de filtrado

In [ ]:
def filtrar_sin_consentimiento(data: Dict[str, Any], forms: Dict[str, List[Dict[str, Any]]]) -> bool:
    pid = data.get('participante_id')
    if not pid:
        return False
    consent_form_id = os.getenv('ODK_FORM_CONSENT', 'Laura2-piloto-encuesta-ic')
    consent_subs = forms.get(consent_form_id, [])
    for sub in consent_subs:
        if sub.get('preamble', {}).get('part_id') == pid:
            return False
    return True

## 3. Enviar correos a los participantes filtrados

In [ ]:
DEFAULT_SUBJECT = 'Proyecto Laura - Notificacion'
DEFAULT_MESSAGE = (
    '<p>Hola,</p><p>Este es un mensaje generado automaticamente desde el entorno local del proyecto Laura.</p>'
)

filtered = []
for entity in participants:
    data = entity.get('currentVersion', {}).get('data', {})
    if filtrar_sin_consentimiento(data, forms_data):
        filtered.append(entity)
        email = data.get('correo') or data.get('email')
        if email:
            await send_email(DEFAULT_SUBJECT, DEFAULT_MESSAGE, email)

print(f'Correos enviados a {len(filtered)} participantes')